In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

root_dir = './09.resource/dogcat_data/'
target_train_path = root_dir + 'train/'
target_val_path = root_dir + 'val/'

img_height, img_width = 100, 100
nb_train_samples = 2
nb_validation_samples = 1
epochs = 1
batch_size = 1

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
)

val_datagen = ImageDataGenerator(
    rescale=1.0/255,
)

train_generator = train_datagen.flow_from_directory(
    target_train_path,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical',
)


validation_generator = val_datagen.flow_from_directory(
    target_val_path,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical',
)


checkpoint_path = './09.resource/model.h5'
checkpoint = keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor='val_acc',
    verbose=2,
    save_weights_only=False,
    save_best_only=True,
    mode='max',
)

# 在现阶段，tensorflow中已经有很多好用的卷积神经网络的模型，可以直接使用
base_model = keras.applications.VGG16(
    weights='imagenet',
    include_top=False,
)

x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024, activation=keras.activations.relu)(x)
x = keras.layers.Dropout(0.5)(x)

predictions = keras.layers.Dense(2, activation='softmax')(x)



model = keras.Model(
    inputs=base_model.input,
    outputs=predictions,
)

for layer in base_model.layers:
    layer.trainable = False


model.compile(
    optimizer=keras.optimizers.Adam(lr=0.0001, decay=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


Found 78 images belonging to 2 classes.
Found 24 images belonging to 2 classes.


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [9]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples,
    callbacks=[checkpoint],
    verbose=1,
)

24/24 [==============================] - 11s 472ms/step - loss: 0.6760 - acc: 0.4583

Epoch 00001: val_acc improved from -inf to 0.45833, saving model to ./09.resource/model.h5
78/78 [==============================] - 40s 513ms/step - loss: 0.7777 - acc: 0.4487 - val_loss: 0.6760 - val_acc: 0.4583
